In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [2]:
turbo=pd.DataFrame()

In [3]:
for page_num in range(1,100):
    url1=f'https://turbo.az/autos?page={page_num}'  
    page=requests.get(url1)
    html=BeautifulSoup(page.content, 'html.parser')
    cars_info=html.find_all('div', class_='products-i__bottom')
    
    for i in range(len(cars_info)): 
        try:
            information=re.sub('<[^>]+>', ' ', str(cars_info[i]))
            price=int(information.split('  ')[1].strip().replace(' ', ''))
            currency=information.split('  ')[2]
            model=information.split('  ')[4]
            year=information.split('  ')[5].split(' ')[0].replace(',', '')
            engine=information.split('  ')[5].split(',')[1]
            driven=information.split('  ')[5].split(',')[2]
            city=information.split('  ')[6].split(',')[0]
            date=information.split('  ')[6].split(',')[1].strip()
            turbo=turbo.append({
                'Model':model,
                'Production_year':year,
                'Engine':engine,
                'Km_driven': driven,
                'City':city,
                'Date':date,
                'Price':price,
                'Currency':currency
            },ignore_index=True )
        except:
            continue

In [4]:
turbo

,Model,Production_year,Engine,Km_driven,City,Date,Price,Currency
0,Mercedes G 63 AMG,2013,5.5 L,80 000 km,Bakı,11.09.2022 20:17,82500.0,$
1,Chevrolet Malibu,2019,1.5 L,48 000 km,Sumqayıt,dünən 12:24,23500.0,AZN
2,GAZ Biznes 330273,2022,2.7 L,0 km,Bakı,26.08.2022 11:16,43500.0,AZN
3,Kia Stonic,2022,1.4 L,0 km,Bakı,bugün 10:48,37900.0,AZN
4,BMW 528,2011,2.0 L,211 000 km,Bakı,09.09.2022 17:39,27400.0,AZN
...,...,...,...,...,...,...,...,...
3559,KamAz 5410,1989,11 L,187 000 km,Gəncə,dünən 19:25,11300.0,AZN
3560,Chevrolet Cruze,2012,1.4 L,295 000 km,Beyləqan,dünən 19:25,13700.0,AZN
3561,Hyundai i10,2008,1.2 L,186 000 km,Bakı,dünən 19:25,9200.0,AZN
3562,LADA (VAZ) 2106,1993,1.6 L,193 520 km,Sumqayıt,dünən 19:25,3500.0,AZN


In [5]:
url_cur='https://www.cbar.az/'

In [6]:
page1=requests.get(url_cur)

In [7]:
html1=BeautifulSoup(page1.content, 'html.parser')

In [8]:
currencies=html1.find_all('div', class_='relize_item')

In [9]:
curr=re.sub('<[^>]+>', ' ', str(currencies))

In [10]:
curr_usd=float(curr[curr.find('USD')+5:curr.find('1 EUR')].replace(' ', ''))

In [11]:
curr_eur=float(curr[curr.find('EUR')+5:curr.find('1 RUB')].replace(' ', ''))

In [12]:
type(curr_usd)

float

In [13]:
turbo['Price in AZN']=np.where(turbo['Currency']=='$', turbo['Price']*curr_usd, np.where(turbo['Currency']=='€', turbo['Price']*curr_eur, turbo['Price']))

In [14]:
turbo

,Model,Production_year,Engine,Km_driven,City,Date,Price,Currency,Price in AZN
0,Mercedes G 63 AMG,2013,5.5 L,80 000 km,Bakı,11.09.2022 20:17,82500.0,$,140250.0
1,Chevrolet Malibu,2019,1.5 L,48 000 km,Sumqayıt,dünən 12:24,23500.0,AZN,23500.0
2,GAZ Biznes 330273,2022,2.7 L,0 km,Bakı,26.08.2022 11:16,43500.0,AZN,43500.0
3,Kia Stonic,2022,1.4 L,0 km,Bakı,bugün 10:48,37900.0,AZN,37900.0
4,BMW 528,2011,2.0 L,211 000 km,Bakı,09.09.2022 17:39,27400.0,AZN,27400.0
...,...,...,...,...,...,...,...,...,...
3559,KamAz 5410,1989,11 L,187 000 km,Gəncə,dünən 19:25,11300.0,AZN,11300.0
3560,Chevrolet Cruze,2012,1.4 L,295 000 km,Beyləqan,dünən 19:25,13700.0,AZN,13700.0
3561,Hyundai i10,2008,1.2 L,186 000 km,Bakı,dünən 19:25,9200.0,AZN,9200.0
3562,LADA (VAZ) 2106,1993,1.6 L,193 520 km,Sumqayıt,dünən 19:25,3500.0,AZN,3500.0


In [15]:
turbo['Model'].str.split(' ', expand=True)

,0,1,2,3,4
0,Mercedes,G,63,AMG,None
1,Chevrolet,Malibu,None,None,None
2,GAZ,Biznes,330273,None,None
3,Kia,Stonic,None,None,None
4,BMW,528,None,None,None
...,...,...,...,...,...
3559,KamAz,5410,None,None,None
3560,Chevrolet,Cruze,None,None,None
3561,Hyundai,i10,None,None,None
3562,LADA,(VAZ),2106,None,None


In [16]:
turbo['MODEL']=turbo['Model'].str.split(' ', expand=True)[0]

In [17]:
turbo['Model1']=turbo['Model'].str.split(' ', expand=True)[1]

In [18]:
turbo['Model1']=np.where(turbo['Model1'].isna(), '', turbo['Model1'])

In [19]:
turbo['Model2']=turbo['Model'].str.split(' ', expand=True)[2]

In [20]:
turbo['Model2']=np.where(turbo['Model2'].isna(), '', turbo['Model2'])

In [21]:
turbo['Brand']=turbo['Model1']+' '+turbo['Model2']

In [22]:
turbo['Model1']

0            G
1       Malibu
2       Biznes
3       Stonic
4          528
         ...  
3559      5410
3560     Cruze
3561       i10
3562     (VAZ)
3563     (VAZ)
Name: Model1, Length: 3564, dtype: object

In [23]:
turbo['Model2']

0           63
1             
2       330273
3             
4             
         ...  
3559          
3560          
3561          
3562      2106
3563      2106
Name: Model2, Length: 3564, dtype: object

In [24]:
turbo=turbo.drop(['Model','Model1', 'Model2'], axis=1)

In [25]:
turbo=turbo.rename(columns={'MODEL':'Model'})

In [26]:
turbo

,Production_year,Engine,Km_driven,City,Date,Price,Currency,Price in AZN,Model,Brand
0,2013,5.5 L,80 000 km,Bakı,11.09.2022 20:17,82500.0,$,140250.0,Mercedes,G 63
1,2019,1.5 L,48 000 km,Sumqayıt,dünən 12:24,23500.0,AZN,23500.0,Chevrolet,Malibu
2,2022,2.7 L,0 km,Bakı,26.08.2022 11:16,43500.0,AZN,43500.0,GAZ,Biznes 330273
3,2022,1.4 L,0 km,Bakı,bugün 10:48,37900.0,AZN,37900.0,Kia,Stonic
4,2011,2.0 L,211 000 km,Bakı,09.09.2022 17:39,27400.0,AZN,27400.0,BMW,528
...,...,...,...,...,...,...,...,...,...,...
3559,1989,11 L,187 000 km,Gəncə,dünən 19:25,11300.0,AZN,11300.0,KamAz,5410
3560,2012,1.4 L,295 000 km,Beyləqan,dünən 19:25,13700.0,AZN,13700.0,Chevrolet,Cruze
3561,2008,1.2 L,186 000 km,Bakı,dünən 19:25,9200.0,AZN,9200.0,Hyundai,i10
3562,1993,1.6 L,193 520 km,Sumqayıt,dünən 19:25,3500.0,AZN,3500.0,LADA,(VAZ) 2106


In [27]:
turbo['Engine']=turbo['Engine'].str.split(' ', expand=True)[1]

In [28]:
turbo['Km_driven']=turbo['Km_driven'].str.replace('km', '').str.replace(' ','')

In [29]:
turbo['Km_driven']

0        80000
1        48000
2            0
3            0
4       211000
         ...  
3559    187000
3560    295000
3561    186000
3562    193520
3563    102000
Name: Km_driven, Length: 3564, dtype: object

In [30]:
x_data=pd.get_dummies(turbo)

In [31]:
turbo['Engine']=pd.to_numeric(turbo['Engine'])

In [32]:
turbo['Km_driven']=pd.to_numeric(turbo['Km_driven'])

In [33]:
turbo.dtypes

Production_year     object
Engine             float64
Km_driven            int64
City                object
Date                object
Price              float64
Currency            object
Price in AZN       float64
Model               object
Brand               object
dtype: object

In [34]:
turbo

,Production_year,Engine,Km_driven,City,Date,Price,Currency,Price in AZN,Model,Brand
0,2013,5.5,80000,Bakı,11.09.2022 20:17,82500.0,$,140250.0,Mercedes,G 63
1,2019,1.5,48000,Sumqayıt,dünən 12:24,23500.0,AZN,23500.0,Chevrolet,Malibu
2,2022,2.7,0,Bakı,26.08.2022 11:16,43500.0,AZN,43500.0,GAZ,Biznes 330273
3,2022,1.4,0,Bakı,bugün 10:48,37900.0,AZN,37900.0,Kia,Stonic
4,2011,2.0,211000,Bakı,09.09.2022 17:39,27400.0,AZN,27400.0,BMW,528
...,...,...,...,...,...,...,...,...,...,...
3559,1989,11.0,187000,Gəncə,dünən 19:25,11300.0,AZN,11300.0,KamAz,5410
3560,2012,1.4,295000,Beyləqan,dünən 19:25,13700.0,AZN,13700.0,Chevrolet,Cruze
3561,2008,1.2,186000,Bakı,dünən 19:25,9200.0,AZN,9200.0,Hyundai,i10
3562,1993,1.6,193520,Sumqayıt,dünən 19:25,3500.0,AZN,3500.0,LADA,(VAZ) 2106


In [35]:
turbo['Engine']=pd.to_numeric(turbo['Engine'])

In [36]:
turbo['Production_year']=pd.to_datetime(turbo['Production_year']).dt.year

In [37]:
import sklearn

In [38]:
X_data=turbo.drop(['Date', 'Currency', 'Price', 'Production_year'], axis=1)

In [39]:
X_data

,Engine,Km_driven,City,Price in AZN,Model,Brand
0,5.5,80000,Bakı,140250.0,Mercedes,G 63
1,1.5,48000,Sumqayıt,23500.0,Chevrolet,Malibu
2,2.7,0,Bakı,43500.0,GAZ,Biznes 330273
3,1.4,0,Bakı,37900.0,Kia,Stonic
4,2.0,211000,Bakı,27400.0,BMW,528
...,...,...,...,...,...,...
3559,11.0,187000,Gəncə,11300.0,KamAz,5410
3560,1.4,295000,Beyləqan,13700.0,Chevrolet,Cruze
3561,1.2,186000,Bakı,9200.0,Hyundai,i10
3562,1.6,193520,Sumqayıt,3500.0,LADA,(VAZ) 2106


In [40]:
X_data.isna().any()

Engine          False
Km_driven       False
City            False
Price in AZN    False
Model           False
Brand           False
dtype: bool

In [41]:
X_data['Engine']=X_data['Engine'].fillna(X_data['Engine'].mean())

In [42]:
X_data['Km_driven']=X_data['Km_driven'].fillna(X_data['Km_driven'].mean())

In [43]:
X=pd.get_dummies(X_data).values

In [44]:
Y=turbo['Price in AZN'].values

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, random_state=0)

In [46]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
regressor=RandomForestRegressor()

In [48]:
regressor.fit(X_train, Y_train)

RandomForestRegressor()

In [51]:
prediction=regressor.predict(X_test)

In [52]:
prediction[:5]

array([36540.3 , 20803.5 , 66147.65, 34494.  , 35522.6 ])

In [53]:
Y_test[:5]

array([36600., 20800., 65960., 34500., 35530.])

In [54]:
from sklearn.metrics import r2_score, mean_squared_error

In [55]:
score1=r2_score(Y_test, prediction)

In [56]:
score2=np.sqrt(mean_squared_error(Y_test, prediction))

In [57]:
score1

0.9992940798846581

In [58]:
score2

1422.9751774416302

In [59]:
turbo['Price in AZN'].max()

625600.0

In [61]:
turbo['Price in AZN'].min()

1100.0

### Modeling

In [67]:
X_train=turbo.loc[:,['Engine', 'Km_driven', 'City', 'Model','Brand']].values

In [66]:
Y_train=turbo['Price in AZN'].values

In [68]:
Engine='1.2'
Km_driven=211000
City='Bakı'
Model='Chevrolet'
Brand='Malibu'

In [69]:
test_data=pd.DataFrame()

In [70]:
test_data=test_data.append({
    'Engine':Engine,
    'Km_driven':Km_driven,
    'City':City,
    'Model':Model,
    'Brand':Brand
}, ignore_index=True)

In [71]:
test_data

,Engine,Km_driven,City,Model,Brand
0,1.2,211000.0,Bakı,Chevrolet,Malibu


In [75]:
X_test=test_data.values

In [76]:
regressor.predict(X_test)

ValueError: could not convert string to float: 'Bakı'